We activate the specific "TimeIndependentSchrodingerEquation" environment

In [27]:
using Pkg
Pkg.activate("../")

  Activating project at `~/github_repositories/my_repositories/TimeIndependentSchrodingerEquation`


We need to execute the following commands if we build the environment for the first time

In [28]:
#= Pkg.add("Gridap")
Pkg.add("GridapGmsh")
Pkg.add("Gmsh");
Pkg.add("FileIO");
Pkg.add("LinearAlgebra");
Pkg.add("SparseArrays");
Pkg.add("Arpack"); =#

With this command we can check the intalled packages in the environment

In [29]:
Pkg.status()

Project TimeIndependentSchrodingerEquation v1.0.0-DEV
Status `~/github_repositories/my_repositories/TimeIndependentSchrodingerEquation/Project.toml`
  [7d9fca2a] Arpack v0.5.4
  [5789e2e9] FileIO v1.16.1
  [705231aa] Gmsh v0.2.2
  [56d4f2e9] Gridap v0.17.17
  [3025c34a] GridapGmsh v0.6.1
  [37e2e46d] LinearAlgebra
  [44cfe95a] Pkg v1.9.0
  [2f01184e] SparseArrays


In [30]:
using TimeIndependentSchrodingerEquation
using Gridap,Test

### Check functions inside `MeshGeneratorFunction.jl`

In [119]:
dom2D=(-30.0,30.0,-30.0,30.0);nxy=(100,100);params_model=(dom2D,nxy);
grid_type="Cartesian2D";

In [120]:
model2D=make_model(grid_type,params_model);

In [121]:
#= using Gridap
writevtk(model2d,"model2d") =#

### Check functions inside `BoundaryConditionsFunction.jl`

In [122]:
BC_type="FullDirichlet";
FullDirichlet_values,FullDirichlet_tags=make_boundary_conditions(grid_type,BC_type,ComplexF64);

### Check functions inside `MiscellaneousFunctions.jl`

In [123]:
Ω,dΩ,Γ,dΓ = measures(model2D,3,FullDirichlet_tags)

(BodyFittedTriangulation(), GenericMeasure(), BoundaryTriangulation(), GenericMeasure())

In [124]:
reff = ReferenceFE(lagrangian,Float64,2)

(Lagrangian(), (Float64, 2), Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}())

In [125]:
VSpace,USpace = fe_spaces(model2D,reff,grid_type;BC_type=BC_type,TypeData=ComplexF64)

(UnconstrainedFESpace(), TrialFESpace())

#### We build functions to write a specific Sturm-Liouville formulation

In [126]:
function eigenvalue_problem_functions(params;switch_potential = "QHO_1D")
    ħ::Float64=1.0; m::Float64=1.0;
    if (switch_potential == "QHO_1D")
        # 1D quantum harmonic oscillator potential
        println("Set quantum harmonic oscillator 1D potential");
        ω,x₁=params;
        p_QHO_1D(x) = 0.5*(ħ*ħ)*(1.0/m);                                      # kinetic energy
        q_QHO_1D(x) = 0.5*m*(ω*ω)*(x[1]-x₁)*(x[1]-x₁);                        # 1D harmonic oscillator centered in x₁
        r_QHO_1D(x) = 1.0;
        return p_QHO_1D,q_QHO_1D,r_QHO_1D
    elseif (switch_potential == "QHO_2D")
        # 2D quantum harmonic oscillator potential
        println("Set quantum harmonic oscillator 2D potential");
        ω,x₁,y₁=params;
        p_QHO_2D(x) = 0.5*(ħ*ħ)*(1.0/m);                                       # kinetic energy
        q_QHO_2D(x) = 0.5*m*(ω*ω)*((x[1]-x₁)*(x[1]-x₁)+(x[2]-y₁)*(x[2]-y₁));   # 2D harmonic osicllator centered in (x₁,y₁)
        r_QHO_2D(x) = 1.0;
        return p_QHO_2D,q_QHO_2D,r_QHO_2D
    end
end

eigenvalue_problem_functions (generic function with 1 method)

In [127]:
p,q,r = eigenvalue_problem_functions((1.0,0.0,0.0);switch_potential = "QHO_2D")

Set quantum harmonic oscillator 2D potential


(var"#p_QHO_2D#50"{Float64, Float64}(1.0, 1.0), var"#q_QHO_2D#51"{Float64, Float64}(0.0, Core.Box(0.0), Core.Box(1.0), 1.0), var"#r_QHO_2D#52"())

### Check functions inside `EigenProblemSolveFunction.jl`

In [128]:
ϵ,ϕ = eigen_values_and_eigen_vectors(p,q,r,dΩ,USpace,VSpace;params=(4,10e-9,500,:none,0.0))

(ComplexF64[1.0006454857510692 + 6.371202552116489e-17im, 2.0025198370964614 + 1.3913431228188513e-17im, 2.002519837096465 + 5.4346795319831805e-17im, 3.0043941884418626 + 1.1076483977622532e-16im], CellField[SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction(), SingleFieldFEFunction()])

#### We build an array with exactly eigenenergie values to quantum 2D harmonic oscillator

In [129]:
function exactly_eigenvalues_2dqho(num_eigval::Integer)
    ϵ_real_aux=Array{Float64}(undef, num_eigval^2)
    index::Int64=1
    for i in 1:num_eigval
        for j in 1:num_eigval
            ϵ_real_aux[index]=((i-1)+(j-1)+1)
            index+=1
        end
    end
    ϵ_real_aux=sort(ϵ_real_aux)
    ϵ_real = ϵ_real_aux[1:num_eigval];
    return ϵ_real
end

exactly_eigenvalues_2dqho (generic function with 1 method)

In [130]:
ϵ_real=exactly_eigenvalues_2dqho(length(ϵ))

4-element Vector{Float64}:
 1.0
 2.0
 2.0
 3.0

In [131]:
@test real(ϵ) ≈ ϵ_real atol=0.1

Test Passed

#### We define useful functions to check the eigenvectors

In [132]:
function norm_l2(𝜳::CellField,dΩ::Gridap.CellData.GenericMeasure)
    return sqrt(real(sum(∫(𝜳'*𝜳)*dΩ)));
end

norm_l2 (generic function with 1 method)

In [133]:
@testset "CheckNormalization" begin
    for i in eachindex(ϕ)
        @test norm_l2(ϕ[1],dΩ) ≈ 1.0
    end
end


Test Summary:      | Pass  Total  Time
CheckNormalization |    4      4  0.0s


Test.DefaultTestSet("CheckNormalization", Any[], 4, false, false, true, 1.684440444743038e9, 1.684440444762596e9, false)

In [134]:
function ortho_check_v2(ϕ::Vector{CellField},TrialSpace::FESpace,dΩ::Gridap.CellData.GenericMeasure)
    nev::Integer=length(ϕ)
    OrthoVector=zeros(Float64,round(Int,(nev^2-nev)/2));
    index::Int64=1
    for i in 2:nev
        for j in 1:(i-1)
            OrthoVector[index]=abs(sum(∫(ϕ[j]'*ϕ[i])*dΩ))
            index+=1
        end
    end
    return OrthoVector;
end

ortho_check_v2 (generic function with 1 method)

In [135]:
ortho_check_v2(ϕ,USpace,dΩ)

6-element Vector{Float64}:
 5.220064297156833e-16
 2.701854678449259e-16
 0.061346998503368104
 1.8189745349361495e-16
 3.3402129452651347e-16
 2.8061108949450754e-16

In [136]:
@testset "Check orthogonalization" begin
    for i in eachindex(ϕ)
        @test Float64(ortho_check_v2(ϕ,USpace,dΩ)[i]) ≈ zero(Float64) atol=Float64(0.1)
    end
end

Test Summary:           | Pass  Total  Time
Check orthogonalization |    4      4  0.1s


Test.DefaultTestSet("Check orthogonalization", Any[], 4, false, false, true, 1.684440444952778e9, 1.684440445024201e9, false)